# OpenAI's CartPole 

In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import median, mean
from collections import Counter

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
LR = 1e-3
env = gym.make("MountainCar-v0")
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 500

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
def initial():
    
    scores=[]
    accepted_score=[]
    training_data=[]
    
    for i in range(initial_games):
        memory =[]
        score= 0 
        #previous_observation= []
        
        for i in range(goal_steps):
            action = random.randrange(0,2)
            observation, reward, info, done = env.step(action=action)
            
            if len(observation)>0:
                memory.append([observation,action])
            score +=reward
            
            if done:
                break
            
            for data in memory:
                if data[1]==1:
                    output= [1,0]
                elif data[1]==0:
                        output =[0,1]
            training_data.append([data[0],output])
            env.reset()
            scores.append(score)
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    
    #print('Average accepted score:',mean(accepted_score))
    return training_data                

In [4]:
initial()

[[array([-0.56381586, -0.00070435]), [0, 1]],
 [array([-5.09069107e-01, -1.09760423e-04]), [1, 0]],
 [array([-5.93400412e-01,  5.23534704e-04]), [1, 0]],
 [array([-4.72408991e-01, -3.85271390e-04]), [1, 0]],
 [array([-5.85321232e-01,  4.63695429e-04]), [1, 0]],
 [array([-0.40331522, -0.00189597]), [0, 1]],
 [array([-0.52497949, -0.00099712]), [0, 1]],
 [array([-0.44016653, -0.00062376]), [1, 0]],
 [array([-5.88784040e-01, -5.18032366e-04]), [0, 1]],
 [array([-5.8418482e-01, -5.5217616e-04]), [0, 1]],
 [array([-0.56561107, -0.00069088]), [0, 1]],
 [array([-4.91484542e-01, -2.42293033e-04]), [1, 0]],
 [array([-5.55532498e-01,  2.40935396e-04]), [1, 0]],
 [array([-0.50900758, -0.00111777]), [0, 1]],
 [array([-0.57959776, -0.00058632]), [0, 1]],
 [array([-0.43736373, -0.00165154]), [0, 1]],
 [array([-4.84830010e-01, -2.92287887e-04]), [1, 0]],
 [array([-5.02087277e-01, -1.62440023e-04]), [1, 0]],
 [array([-0.4561114 , -0.00050642]), [1, 0]],
 [array([-5.96247902e-01,  5.44551842e-04]), [1,

In [5]:
def neural_network_model(input_size):

    network = input_data(shape=[None, input_size, 1], name='input')

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network)

    return model

'''def train_model(training_data,model=False):
    
    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    Y = [i[1] for i in training_data]
    if not model:
        model = neural_network_model(input_size= len(X[0]))
    model.fit({'input': X}, {'targets': Y}, n_epoch=2, snapshot_step=500, show_metric=True) 
    return model'''
def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]

    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input': X}, {'targets': y}, n_epoch=3, snapshot_step=500, show_metric=True, run_id='openai_learning')
    return model

In [6]:
training_data = initial()

In [7]:
model = train_model(training_data)

Training Step: 11720  | total loss: 0.69321 | time: 49.081s
| Adam | epoch: 003 | loss: 0.69321 - acc: 0.4995 -- iter: 249984/250000
Training Step: 11721  | total loss: 0.69318 | time: 49.093s
| Adam | epoch: 003 | loss: 0.69318 - acc: 0.5011 -- iter: 250000/250000
--


In [8]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        env.render()

        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            #print(model.predict(prev_obs.reshape(-1,len(prev_obs),1)))
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1)))

        choices.append(action)
                
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score+=reward
        if done: break

    scores.append(score)

print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
print(score_requirement)


Average Score: -200.0
choice 1:0.9995  choice 0:0.0005
50
